To do:
- ADD WEIGHTED LOSS FUNCTION FOR HITS ON 1 FOR POSITIVE LABELLING AS THIS IS WAY MORE RARE = HIGHER "REWARD"
- Have a look at learning rate and gradient norm clipping which I need to read up on.
    - In addition to this, look at the implications of gradient accumulation steps
    - Much of this pipeline was constrained due to computational restrictions which I think was caused by errors and not actual training process.
- Early stopping: load_best_model_at_end=True
- Split into validation set as well (80:10:10)
- Hyperparameter tuning (Alpha, learning rate, batch size so on - not sure how to figure this out)
    - There is precedence for no hyperparameter tuning from the author of the OG NLI model that DEBATE is based on = Due to computational restrains and the points from this paper, no hyperparameter tuning was performed in this case. The model tuning in itself is also not the primary focus in this paper, but simply serves as a tool for the actual inquiry into blame in the Danish Parliament
- Context / No context?
    - Most likely no context, as we do not have the academic reasons for doing so 
    - Model actually also seems to perform better without (f1 of .66 for no context on random test set vs. . 55 on gold labelled for context model) This might also make sense as the context is not taken into account in the DEBATE model


In [1]:
#%pip install -r "requirements_bert.txt"

In [2]:
import torch
import transformers
import bitsandbytes
import accelerate
import datasets
import numpy as np
import pandas as pd
import keras
import json
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from sklearn.model_selection import train_test_split
from datasets import Dataset
from keras.losses import binary_crossentropy
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, recall_score, classification_report, confusion_matrix

/work/MarkusLundsfrydJensen#1865/miniconda3/envs/blame_bert/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-29 14:32:42.088896: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-29 14:32:42.138503: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-29 14:32:43.107162: I tensorflow/core/util/port.cc:153] oneD

In [3]:
base_model_name = "jhu-clsp/mmBERT-base"

base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    num_labels=2,
    torch_dtype=torch.bfloat16, #b addded
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at jhu-clsp/mmBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
lora_config = LoraConfig(
    r=16,  # Low-rank dimension
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules="all-linear"  # Fine-tuning all linear (classification, attention... layers)
)

lora_model = prepare_model_for_kbit_training(base_model)

# Enable gradient computation for LoRA parameters
for name, param in lora_model.named_parameters():
    if 'lora' in name.lower():
        param.requires_grad = True

lora_model = get_peft_model(lora_model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 3,416,096 || all params: 310,947,874 || trainable%: 1.0986


In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], 
    padding="max_length", 
    truncation=True,
    max_length=512, # Padding to 512 to massively cut down on computation compared to base 8,192 tokens. 
    )

In [6]:
val_dataframe = pd.read_json("/work/MarkusLundsfrydJensen#1865/Bachelor_project/Model_data/validation_set.json")

val_dataframe = val_dataframe[['text', 'label']]

val_dataframe.rename(columns={'label': 'labels'}, inplace=True)

val_dataset = Dataset.from_pandas(val_dataframe)
tokenized_val = val_dataset.map(tokenize_function, batched=True, num_proc=16)


Map (num_proc=16): 100%|██████████| 258/258 [00:00<00:00, 554.27 examples/s]


In [7]:
input_data_dir = "/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data_3_4_5_temps.json"

dataframe_3_4_5 = pd.read_json(input_data_dir)

dataframe_3_4_5 = dataframe_3_4_5.sample(frac=1)

test_dataframe = dataframe_3_4_5[['text', 'label']]

test_dataframe.rename(columns={'label': 'labels'}, inplace=True)

test_dataframe = test_dataframe[0:50000]

test_dataset = Dataset.from_pandas(test_dataframe)

tokenized_test = test_dataset.map(tokenize_function, batched=True, num_proc=16)

/tmp/ipykernel_14772/775643221.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataframe.rename(columns={'label': 'labels'}, inplace=True)
Map (num_proc=16): 100%|██████████| 50000/50000 [00:03<00:00, 12907.84 examples/s]


In [8]:
# 1. Smaller batch size with gradient accumulation
training_args = TrainingArguments(
    output_dir='/work/MarkusLundsfrydJensen#1865',
    #optim="paged_adamw_8bit",
    learning_rate=1e-4,
    num_train_epochs=5,
    per_device_train_batch_size=16,  # Reduced
    gradient_accumulation_steps=16,  # Effective batch = 256
    logging_steps=1,  # More frequent logging
    eval_strategy="epoch",  # Evaluate more frequently
    save_strategy="epoch",
    bf16=True,
    fp16=False,
    dataloader_pin_memory=True,
    dataloader_num_workers=8,
    remove_unused_columns=True,
    max_grad_norm=1.0,
    disable_tqdm=False,
    load_best_model_at_end=True,  # Add this
    metric_for_best_model="weighted BCE", # Add this
    greater_is_better = False
      
)

#OBS implement early stopping and more epochs, lower learning rate?

In [9]:
def weighted_bincrossentropy(true, pred, weight_zero = 1, weight_one = 1):
    """
    Calculates weighted binary cross entropy. The weights are fixed to represent class imbalance in the dataset.
        
    For example if there are 10x as many positive classes as negative classes,
        if you adjust weight_zero = 1.0, weight_one = 0.1, then false positives 
        will be penalized 10 times as much as false negatives.

    """
  
    # calculate the binary cross entropy
    bin_crossentropy = binary_crossentropy(true, pred)
    
    # apply the weights
    weights = true * weight_one + (1. - true) * weight_zero
    #weights /= (weight_one + weight_zero) # Normalizing to be more consistent with regular BCE for comparison 
    weighted_bin_crossentropy = weights * bin_crossentropy 

    return np.mean(weighted_bin_crossentropy)

In [10]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    #From logits to probabilities
    probs_2d = np.exp(predictions) / np.exp(predictions).sum(axis=1, keepdims=True)
    probs = probs_2d[:, 1]  # positive class extraction
    
    weigthted_bce = weighted_bincrossentropy(labels, probs)
    keras_bce = binary_crossentropy(labels, probs)
    keras_bce = float(np.mean(keras_bce.numpy()))  # Converting from keras eagertensor to float value
    
    # Wrapping all metrics to floats for json serialization during model eval
    return {
        'keras_BCE': keras_bce,
        'weighted BCE': weigthted_bce,
        'recall': float(recall_score(labels, probs.round())),
        'precision': float(average_precision_score(labels, probs)),
        'accuracy': float(accuracy_score(labels, probs.round())), # Need rounding for these two computations (integer required)
        'f1': float(f1_score(labels, probs.round(), average='macro')), # macro f1 is better for imbalanced dataset
        'number_of_true_preds': sum(probs.round()),
        'number_of_true_labels': sum(labels)
    }

In [11]:
# Create custom trainer with weighted loss

from torch import nn
import torch

# Calculate class weights
label_counts = test_dataframe['labels'].value_counts()
total = len(test_dataframe)
weight_for_0 = total / (2 * label_counts[0])
weight_for_1 = total / (2 * label_counts[1])

print(f"Class 0 weight: {weight_for_0}")
print(f"Class 1 weight: {weight_for_1}")
print(f"Label distribution: {label_counts}")

class WeightedLossTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(model.device))
        loss = loss_fct(logits, labels)
        
        return (loss, outputs) if return_outputs else loss

Class 0 weight: 0.5047038397868131
Class 1 weight: 53.648068669527895
Label distribution: labels
0    49534
1      466
Name: count, dtype: int64


In [ ]:
# Create the class weights tensor
class_weights = torch.tensor([weight_for_0, weight_for_1], dtype=torch.float32)
print(class_weights)
# Use the custom trainer instead of the regular Trainer
trainer = WeightedLossTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_test,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
    class_weights=class_weights  # Pass the weights here
)

trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.


tensor([ 0.5047, 53.6481])


Epoch,Training Loss,Validation Loss


In [ ]:
# Merge (now clean, no quantization artifacts)
model = trainer.model
merged_model = model.merge_and_unload()

output_model_dir = "/work/MarkusLundsfrydJensen#1865/test_models_save/Option_2"
merged_model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

('/work/MarkusLundsfrydJensen#1865/test_models_save/Option_2/tokenizer_config.json',
 '/work/MarkusLundsfrydJensen#1865/test_models_save/Option_2/special_tokens_map.json',
 '/work/MarkusLundsfrydJensen#1865/test_models_save/Option_2/tokenizer.json')

In [ ]:
#Hold up

In [16]:
#OBS need to be made usable
class BlameDetectorDa_v0(object):

    def __init__(self, model, tokenizer, max_length):

        self.model = model
        self.max_length = max_length
        self.tokenizer = tokenizer
        #self.batch_size = batch_size

        self.model_initialization()

        return

    def model_initialization(self):
        #self.tokenizer = AutoTokenizer.from_pretrained(self.tokenizer_path)
        self.model.eval()
            
        # Move to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(self.device)
        self.model.to(self.device)

        print(f"Model loaded successfully on {self.device}")

        return

    def predict(self):
        """Make a prediction on a single text input."""
        # Tokenize input
        inputs = self.tokenizer(
            self.text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Move inputs to device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Make prediction
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0][predicted_class].item()
        
        return predicted_class, confidence, probabilities[0].cpu().numpy()

    def run_prediction(self, text):

        self.text = text
        predicted_class, confidence, probs = self.predict()
            
        return predicted_class, confidence


In [18]:
BD = BlameDetectorDa_v0(model = model_tester, tokenizer = tokenizer, max_length = 512)

cuda
Model loaded successfully on cuda


In [19]:
import json
json_path = "/work/MarkusLundsfrydJensen#1865/Bachelor_project/Model_data/validation_set.json"

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [21]:
for entry in data:
    text = entry["text"]

    entry["predicted_class"], entry["confidence"] = BD.run_prediction(text)

In [22]:
true_labels = 0
pred_true = 0
false_labels = 0
pred_false = 0

correct_pred = 0
incorrect_pred = 0

for entry in data:
    if entry["label"] == 1:
        true_labels += 1
    if entry["predicted_class"] == 1:
        pred_true +=1

    if entry["label"] == 0:
        false_labels += 1
    if entry["predicted_class"] == 0:
        pred_false +=1

    if entry["label"] == entry["predicted_class"]:
        correct_pred +=1

    if entry["label"] != entry["predicted_class"]:
        incorrect_pred +=1


print(correct_pred/len(data))

0.7558139534883721
